# VALUE AT RISK

Abstract Focusing on the creation of portfolios for investment, this chapter aims to understand the risks of the portfolio through methods such as the Value at Risk (VaR) to determine the possible loss or gain of a portfolio. This chapter is based on an investor view and the process for executing decisions that create profitable portfolios in the short and long run.

Keywords Risk · Portfolios · VaR · Backtesting

The concept of Value at Risk (VaR) is one of the most interesting in finance because it analyzes the maximum loss that a portfolio may have (Damodaran 2018). This is another measure of risk that deserves to be separated from portfolio and risk because of the difference that it has with the ratios (Sharpe, Traynor, Information and Jensen) in the previous chapter. To summarize the VaR, it gives the worst loss on a certain time horizon based on the confidence level assigned to the model.

## Historical VAR(95)

Since the VaR is based on the confidence level, it may have different results based on a 65%, 90%, 95% or any other confidence interval. The following example is Historical VaR(95), meaning that the confidence interval will be at a 95%.


In [155]:
import numpy as np
import pandas as pd
import yfinance as yf
import datetime
import matplotlib.pyplot as plt
from scipy.stats import norm

%matplotlib inline


### • Choose the portfolio

In [156]:
start=datetime.datetime(2018,1,2)
end=datetime.datetime(2019,4,1)

tickers=['AAPL','WMT','TM','KO','BA']

stocks=pd.DataFrame()
for x in tickers:
    stocks[x]=yf.download(tickers=x,start=start,end=end)['Close']

stocks.tail()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,AAPL,WMT,TM,KO,BA
Date,,,,,
2019-03-25,47.185001,98.169998,120.489998,46.029999,370.459991
2019-03-26,46.697498,98.320000,121.769997,46.639999,370.380005
2019-03-27,47.117500,97.209999,121.029999,46.610001,374.209991
2019-03-28,47.180000,97.129997,119.070000,46.580002,374.440002
2019-03-29,47.487499,97.529999,118.019997,46.860001,381.420013


### • Calculate the returns

In [157]:
stocks_return=np.log((stocks/stocks.shift(1)))
stocks_return.tail()

,AAPL,WMT,TM,KO,BA
Date,,,,,
2019-03-25,-0.012165,-0.001120,0.002243,0.002175,0.022632
2019-03-26,-0.010385,0.001527,0.010567,0.013165,-0.000216
2019-03-27,0.008954,-0.011354,-0.006096,-0.000643,0.010288
2019-03-28,0.001326,-0.000823,-0.016327,-0.000644,0.000614
2019-03-29,0.006496,0.004110,-0.008857,0.005993,0.018470


### • Assign random portfolio weights that sum to one (1)

In [158]:
portfolio_weights = np.array(np.random.random(5))
portfolio_weights
portfolio_weights=portfolio_weights/np.sum(portfolio_weights)
portfolio_weights


array([0.38997558, 0.18533178, 0.0283654 , 0.28352394, 0.1128033 ])

This step is interesting because, in the Portfolio and Risk chapter, the purpose was to assign the same return to each of the stocks. In this case the np.random.random creates weights for the five (5) stocks but it often gives a number less or higher than 100%. Therefore it has to be balanced by dividing the weights in the sum to obtain a portfolio that sums 100%.

### • Multiply the portfolio with the stocks

In [159]:
weighted_returns_portfolio = stocks_return.mul(portfolio_weights,axis=1)
weighted_returns_portfolio.tail()

,AAPL,WMT,TM,KO,BA
Date,,,,,
2019-03-25,-0.004744,-0.000208,0.000064,0.000617,0.002553
2019-03-26,-0.004050,0.000283,0.000300,0.003733,-0.000024
2019-03-27,0.003492,-0.002104,-0.000173,-0.000182,0.001160
2019-03-28,0.000517,-0.000153,-0.000463,-0.000183,0.000069
2019-03-29,0.002533,0.000762,-0.000251,0.001699,0.002083


###• Convert returns to percentages and drop the missing values

In [160]:
stocks_return['Portfolio'] = weighted_returns_portfolio.sum(axis=1).dropna()
stocks_return['Portfolio'] = stocks_return['Portfolio'] * 100
stocks_return.tail()

,AAPL,WMT,TM,KO,BA,Portfolio
Date,,,,,,
2019-03-25,-0.012165,-0.001120,0.002243,0.002175,0.022632,-0.171833
2019-03-26,-0.010385,0.001527,0.010567,0.013165,-0.000216,0.024092
2019-03-27,0.008954,-0.011354,-0.006096,-0.000643,0.010288,0.219272
2019-03-28,0.001326,-0.000823,-0.016327,-0.000644,0.000614,-0.021198
2019-03-29,0.006496,0.004110,-0.008857,0.005993,0.018470,0.682650


### • Calculate the VaR95

In [161]:
var95=np.percentile(stocks_return['Portfolio'], 5)
var95

-2.021283218289118

Based on the historical returns of the portfolio at a 95% confidence interval, the worst loss is a var95 loss, therefore the result is negative.

## Historical VAR(99)

For computing the Historical VaR at a 99% confidence level the only change that has to be done is in the last part of the script, changing the np.percentile to 1, which means the 1%.

In [162]:
var99=np.percentile(stocks_return['Portfolio'], 1)
var99

-3.1657969556905954

At a 99% confidence level the worst loss is var99 with the portfolio. Clearly the VaR is higher given that the confidence level is lower. This is rational and therefore it helps understand the process by which the VaR works, given that a higher confidence level will give a higher percentage of loss and a lower confidence level will give a lower percentage of loss.

## VAR FOR THE NEXT 10 DAYS

One of the most important aspects of calculating a VAR is to calculate the effect on the investment in terms of money. As far, the VAR model has centered on the percentage loss, but for the next example the process is to analyze the VAR if USD 1 million is invested. For example, the same data set will be used.

### • Choose the portfolio

In [163]:
start=datetime.datetime(2018,1,2)
end=datetime.datetime(2019,4,1)

tickers=['AAPL','WMT','TM','KO','BA']

stocks=pd.DataFrame()

for x in tickers:
    stocks[x]=yf.download(tickers=x,start=start,end=end)['Close']

stocks.tail()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,AAPL,WMT,TM,KO,BA
Date,,,,,
2019-03-25,47.185001,98.169998,120.489998,46.029999,370.459991
2019-03-26,46.697498,98.320000,121.769997,46.639999,370.380005
2019-03-27,47.117500,97.209999,121.029999,46.610001,374.209991
2019-03-28,47.180000,97.129997,119.070000,46.580002,374.440002
2019-03-29,47.487499,97.529999,118.019997,46.860001,381.420013


### • Calculate the returns

In [164]:
stocks_return=np.log((stocks/stocks.shift(1)))
stocks_return.tail()

,AAPL,WMT,TM,KO,BA
Date,,,,,
2019-03-25,-0.012165,-0.001120,0.002243,0.002175,0.022632
2019-03-26,-0.010385,0.001527,0.010567,0.013165,-0.000216
2019-03-27,0.008954,-0.011354,-0.006096,-0.000643,0.010288
2019-03-28,0.001326,-0.000823,-0.016327,-0.000644,0.000614
2019-03-29,0.006496,0.004110,-0.008857,0.005993,0.018470


### • Assign random portfolio weights that sum to one (1)

In [165]:
portfolio_weights = np.array(np.random.random(5))
portfolio_weights
portfolio_weights=portfolio_weights/np.sum(portfolio_weights)
portfolio_weights

array([0.02478321, 0.22097733, 0.35051365, 0.04556821, 0.3581576 ])

### • Multiply the portfolio with the stocks

In [166]:
weighted_returns_portfolio = stocks_return.mul(portfolio_weights,axis=1)

weighted_returns_portfolio.tail()

,AAPL,WMT,TM,KO,BA
Date,,,,,
2019-03-25,-0.000301,-0.000247,0.000786,0.000099,0.008106
2019-03-26,-0.000257,0.000337,0.003704,0.000600,-0.000077
2019-03-27,0.000222,-0.002509,-0.002137,-0.000029,0.003685
2019-03-28,0.000033,-0.000182,-0.005723,-0.000029,0.000220
2019-03-29,0.000161,0.000908,-0.003105,0.000273,0.006615


### • Calculate the returns based on the weights

In [167]:
stocks_return['Portfolio']=weighted_returns_portfolio.sum(axis=1).dropna()
stocks_return.tail()

,AAPL,WMT,TM,KO,BA,Portfolio
Date,,,,,,
2019-03-25,-0.012165,-0.001120,0.002243,0.002175,0.022632,0.008442
2019-03-26,-0.010385,0.001527,0.010567,0.013165,-0.000216,0.004307
2019-03-27,0.008954,-0.011354,-0.006096,-0.000643,0.010288,-0.000768
2019-03-28,0.001326,-0.000823,-0.016327,-0.000644,0.000614,-0.005681
2019-03-29,0.006496,0.004110,-0.008857,0.005993,0.018470,0.004853


### • Determine the average (mu) of the returns

In [168]:
mu=stocks_return['Portfolio'].mean()
mu

0.00019764062257805997

### • Determine the standard deviation (sigma) of the returns

In [169]:
sigma=stocks_return['Portfolio'].std()
sigma

0.011697673571734728

### • Assign a confidence level to the VaR (99% for this example)

In [170]:
confidence=0.99
confidence

0.99

### • Calculate the alpha

In [171]:
alpha=norm.ppf(1-confidence)
alpha

-2.3263478740408408

For this example, the norm.ppf is being used, the reason for this is that it determines the probability density function of one (1) minus the confidence interval. This is useful because it determines the probability of the VAR. It is a similar process to the np.percentile.

### • Create a position


In [172]:
position=1e6
position

1000000.0

The position is the investment on the portfolio. Since the portfolio was created, in this case the investment is USD 1 million. The interesting aspect of using 1e6 for a million is to include a complex number structure that is easier to write. The other choice would have been to write the 1,000,000.

### • Calculate the VaR
